In [82]:
from typing import Dict, DefaultDict, Set, List, Tuple
from collections import defaultdict
import math
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow

from structs import Item, Recipe

import json

class RecipeGraph:
    """
    Represents the recipe relationships as a directed graph.

    Attributes:
        nodes (Dict[str, Item]): Dictionary of Item objects in the graph, keyed by item name.
        edges (DefaultDict[Item, Dict[Item, float]]): Adjacency list representing the graph's edges.
                                                    The keys are Item objects (inputs), and the values are 
                                                    dictionaries mapping output Item objects to edge weights 
                                                    (output quantity per input quantity).
        recipes_by_output (Dict[Item, List[Recipe]]): Dictionary mapping Item objects to a list of recipes 
                                                      that produce that item.
    """

    def __init__(self):
        self.nodes: Dict[str, Item] = {}
        self.edges: DefaultDict[Item, Dict[Item, float]] = defaultdict(dict)
        self.recipes_by_output: Dict[Item, Set[Recipe]] = defaultdict(list)

    def add_recipe(self, recipe: Recipe) -> None:
        """
        Adds a recipe to the graph.
        """
        for inp in recipe.inputs:
            input_item = self._get_or_create_node(inp.item.itemName)
            for out in recipe.outputs:
                output_item = self._get_or_create_node(out.item.itemName)
                # Calculate edge weight (output quantity per 1 input quantity)
                edge_weight = out.quantity / inp.quantity  
                self.edges[input_item][output_item] = edge_weight
                if recipe not in self.recipes_by_output[output_item]:
                    self.recipes_by_output[output_item].append(recipe)

    def _get_or_create_node(self, item_name: str) -> Item:
        """
        Gets an existing node (Item) or creates a new one if it doesn't exist.
        """
        if item_name not in self.nodes:
            self.nodes[item_name] = Item(item_name)
        return self.nodes[item_name]


class RecipeLoader:
    """
    Loads recipes from a JSON file and constructs a RecipeGraph.

    Attributes:
        graph (RecipeGraph): The recipe graph.
    """

    def __init__(self, recipes_file: str = "recipes.json"):
        """
        Initializes the RecipeLoader by loading recipes from the specified JSON file.
        """
        self.graph = RecipeGraph()
        self._load_recipes(recipes_file)

    def max_flow(self, source_items: Dict[Item, float], sink_item: Item) -> Dict[Item, float]:
        """
        Calculates the maximum flow from source items to a sink item in the recipe graph 
        using scipy.sparse.csgraph.maximum_flow.

        Args:
            source_items (Dict[Item, float]): A dictionary of source items with their available quantities.
            sink_item (Item): The sink item (the target output).

        Returns:
            Dict[Item, float]: A dictionary of maximum output quantities for each item.
        """
        source = Item("source")
        sink = Item("sink")

        graph, node_indices = self._create_flow_graph(source_items, sink_item, source, sink)
        source_index = node_indices[source]
        sink_index = node_indices[sink]

        # Use scipy's maximum_flow
        flow = maximum_flow(csr_matrix(graph), source_index, sink_index)

        # Calculate output quantities from flow values
        output_quantities = {}
        for item, index in node_indices.items():
            rows, cols = flow.flow.nonzero()  
            if (sink_index, index) in zip(rows, cols):
                output_quantities[item] = flow.flow[sink_index, index]

        return output_quantities

    def _create_flow_graph(self, source_items: Dict[Item, float], sink_item: Item, source: Item, sink: Item) -> Tuple[np.ndarray, Dict[Item, int]]:
        """
        Creates a numpy array representing the flow graph with source and sink nodes.
        """
        
        node_indices = {item: i for i, item in enumerate(self.graph.nodes.values())}
        node_indices[source] = len(node_indices)  
        node_indices[sink] = len(node_indices)
        num_nodes = len(node_indices)

        graph = np.zeros((num_nodes, num_nodes), dtype=int)

        for input_item, outputs in self.graph.edges.items():
            for output_item, edge_weight in outputs.items():  # Corrected line
                input_index = node_indices[input_item]
                output_index = node_indices[output_item]
                graph[input_index, output_index] += int(edge_weight)

        # Add source node with edges to source items
        source_index = node_indices[source]
        for item, quantity in source_items.items():
            item_index = node_indices[item]
            graph[source_index, item_index] = int(quantity)

        # Add sink node with edges from items that can produce the sink item
        sink_index = node_indices[sink]
        for input_item, outputs in self.graph.edges.items():
            if sink_item in outputs:
                input_index = node_indices[input_item]
                graph[input_index, sink_index] = int(10**9)

        return graph, node_indices

    def _load_recipes(self, recipes_file: str) -> None:
        """
        Loads recipes from the given JSON file and adds them to the graph.
        """
        with open(recipes_file) as f:
            recipes_json = json.load(f)

        for recipe_data in recipes_json:
            outputs = [
                (i["item"], i["amount"], i["perMin"]) for i in recipe_data["outputs"]
            ]
            inputs = [
                (i["item"], i["amount"], i["perMin"]) for i in recipe_data["inputs"]
            ]

            recipe = Recipe(
                recipe_data["name"],
                recipe_data["time"],
                recipe_data["machine"],
                outputs,
                inputs,
                recipe_data["type"],
            )
            self.graph.add_recipe(recipe)

    def get_recipes_by_type(self, recipe_type: str) -> Dict[str, Recipe]:
        """
        Returns a dictionary of recipes filtered by the specified type.

        Args:
            recipe_type (str): The type of recipes to retrieve.

        Returns:
            Dict[str, Recipe]: A dictionary of recipes matching the given type.
        """
        recipes = {}
        for item, recipes_for_item in self.graph.recipes_by_output.items():
            for recipe in recipes_for_item:
                if recipe.type == recipe_type and recipe.recipeName not in recipes:
                    recipes[recipe.recipeName] = recipe
        return recipes

In [83]:
rm = RecipeLoader()

In [84]:
rm.graph.recipes_by_output[rm.graph.nodes["Motor"]]

[Recipe(recipeName='Motor'),
 Recipe(recipeName='Electric Motor'),
 Recipe(recipeName='Rigor Motor')]

In [92]:
io = rm.graph.nodes["Stator"]
aa = rm.graph.nodes["Stator"]

ip = rm.graph.nodes["Motor"]

In [93]:
rm.max_flow({io: 100.0, aa: 100.0}, ip)

{Item(itemName='Stator'): np.int32(-100)}